In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
import time
import os

In [2]:
# data path
database_path='./profiles/'
archea_full_genome_files = '-mer_all_archea_profiles.csv'
bacteria_full_genome_files = '-mer_all_bacteria_profiles.csv'
archea_genes_files = '-mer_archea_genes.csv'
bacteria_genes_files = '-mer_bacteria_genes.csv'

archea_proteins = '3-mer_archea_protein.csv'
bacteria_proteins = '3-mer_bacteria_protein.csv'

# parameters
k_values = [3, 5, 6]

In [29]:
# 1. loading data, converting to matrix and combining archea/bacteria files
def normalize(dataset):
    for index in dataset.index:
        datas = dataset.loc[index]
        dataset.loc[index] = datas/np.sum(datas)
    return dataset

genome_dataset={}
genes_dataset={}
for k in k_values:
    print(f'Loading data for {k}-mers:')
    
    archea_file = database_path + str(k) + archea_genes_files
    bacteria_file = database_path + str(k) + bacteria_genes_files
    print(archea_file)
    archea_genes = pd.read_csv(archea_file, sep='\t', index_col=0)
    archea_genes = normalize(archea_genes.fillna(0))
    
    print(bacteria_file)
    bacteria_genes = pd.read_csv(bacteria_file, sep='\t', index_col=0)
    bacteria_genes = normalize(bacteria_genes.fillna(0))
    
    genes_data = pd.concat([archea_genes, bacteria_genes])
    genes_dataset[k] = genes_data
    
    archea_file = database_path + str(k) + archea_full_genome_files
    bacteria_file = database_path + str(k) + bacteria_full_genome_files
    print(archea_file)
    archea_genome = pd.read_csv(archea_file, sep='\t', index_col=0)
    archea_genome = normalize(archea_genome.fillna(0))
    
    print(bacteria_file)
    bacteria_genome = pd.read_csv(bacteria_file, sep='\t', index_col=0)
    bacteria_genome = normalize(bacteria_genome.fillna(0))
    
    genome_data = pd.concat([archea_genome, bacteria_genome])
    genome_dataset[k] = genome_data

# computing for proteins
print('Loading data for protein 3-mers')
archea_file = database_path + archea_proteins
bacteria_file = database_path + bacteria_proteins
print(archea_file)
archea_genes = pd.read_csv(archea_file, sep='\t', index_col=0)
archea_genes = normalize(archea_genes.fillna(0))

print(bacteria_file)
bacteria_genes = pd.read_csv(bacteria_file, sep='\t', index_col=0)
bacteria_genes = normalize(bacteria_genes.fillna(0))

protein_data = pd.concat([archea_genes, bacteria_genes]).fillna(0)

Loading data for 3-mers:
./profiles/3-mer_archea_genes.csv
./profiles/3-mer_bacteria_genes.csv
./profiles/3-mer_all_archea_profiles.csv
./profiles/3-mer_all_bacteria_profiles.csv
Loading data for 5-mers:
./profiles/5-mer_archea_genes.csv
./profiles/5-mer_bacteria_genes.csv
./profiles/5-mer_all_archea_profiles.csv
./profiles/5-mer_all_bacteria_profiles.csv
Loading data for 6-mers:
./profiles/6-mer_archea_genes.csv
./profiles/6-mer_bacteria_genes.csv
./profiles/6-mer_all_archea_profiles.csv
./profiles/6-mer_all_bacteria_profiles.csv
Loading data for protein 3-mers
./profiles/3-mer_archea_protein.csv
./profiles/3-mer_bacteria_protein.csv


In [30]:
# k-fold cross validation on different models of kernels
s = 5
C = 100
kernel = 'linear'
scores_genes = []
times_genes = []
scores_genome = []
times_genome = []
y_values = list(np.unique(list(genome_dataset[k_values[0]].index)))

# genomic data
for k in k_values:
    # scoring and timing genes training
    print(f'Computing genes training for {k}-mers')
    X = np.array(genes_dataset[k])
    y = np.array([y_values.index(species.split('.')[0]) for species in genes_dataset[k].index])
    model = svm.SVC(kernel=kernel, C=C)
    k_folds = KFold(n_splits = s)
    time_start = time.time()
    score = cross_val_score(model, X, y)
    time_end = time.time()
    scores_genes += [np.mean(score)]
    times_genes += [time_end- time_start]
    
    # scoring and timing genome training
    print(f'Computing genome training for {k}-mers')
    X_train = np.array(genome_dataset[k])
    y_train = np.array([y_values.index(val) for val in list(genome_dataset[k].index)])
    model = svm.SVC(kernel=kernel, C=C)
    time_start = time.time()
    model.fit(X_train, y_train)
    scores_genome += [np.mean(model.score(X, y))]
    time_end = time.time()
    times_genome += [time_end- time_start]
   
# protein data
print(f'Computing protein training for 3-mers')
X = np.array(protein_data)
y = np.array([y_values.index(species.split('.')[0]) for species in protein_data.index])
model = svm.SVC(kernel=kernel, C=C)
k_folds = KFold(n_splits = s)
time_start = time.time()
score_proteins = np.mean(cross_val_score(model, X, y))
time_end = time.time()
time_proteins = time_end - time_start

Computing genes training for 3-mers
Computing genome training for 3-mers
Computing genes training for 5-mers
Computing genome training for 5-mers
Computing genes training for 6-mers
Computing genome training for 6-mers
Computing protein training for 3-mers


In [34]:
if not os.path.exists('SVM_results'):
    os.mkdir('SVM_results')
results = pd.DataFrame([
    scores_genes,
    scores_genome,
    [score_proteins, np.NaN, np.NaN]
], index = ['Genes training(cross-val)', 'Genome training', 'Protein training(cross-val)'],
columns = k_values)
results.to_csv('SVM_results/accuracies_per_fold.csv')
times = pd.DataFrame([
    times_genes,
    times_genome,
    [time_proteins, np.NaN, np.NaN]
], index = ['Genes training(cross-val)', 'Genome training', 'Protein training(cross-val)'],
columns = k_values)
times.to_csv('SVM_results/execution_times.csv')